In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as ss

In [2]:
from statsmodels.formula.api import mixedlm

In [3]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', None)

In [4]:
# #get rid of scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [5]:
data = pd.read_csv("data_pos_02_latency_backfilled_happiness.csv")


In [6]:
# calculate fatigue
data["tap_count"] = [1] * len(data)
tap_count = data.groupby(['Subject', 'trial_label'])['tap_count'].cumsum().reset_index(name = "fatigue")
tap_count = tap_count.reset_index().set_index("index")
tap_count = tap_count.drop("level_0", axis = 1)


In [7]:
data = tap_count.merge(data, left_index = True, right_index = True, how = "inner")


In [8]:
data["log_latency"] = np.log(data["latency"])


In [9]:
data.price_displayed.unique()


array([0.4, 0.1, 2.5, 1. , nan])

In [10]:
data.task.unique()

array(['fishing_pos', 'mood_rating_pos'], dtype=object)

In [11]:
# drop mood rating trials
data = data[data.task == 'fishing_pos']

In [12]:
data.latency.isnull().values.any()
data = data[data.latency.notnull()]

In [13]:
f_model = mixedlm(formula = "log_latency ~ 1 + response + price_displayed + fatigue",
                 groups = data["Subject"],
                 re_formula = "~ response + price_displayed + fatigue", data = data)
r = f_model.fit(reml = False, method = "powell")


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2094: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [14]:
print (r.summary())

                  Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      log_latency
No. Observations:       40130        Method:                  ML         
No. Groups:             29           Scale:                   0.0587     
Min. group size:        899          Likelihood:              -237.3747  
Max. group size:        1590         Converged:               Yes        
Mean group size:        1383.8                                           
-------------------------------------------------------------------------
                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                       5.200    0.081 64.150 0.000  5.041  5.359
response                       -0.001    0.001 -0.999 0.318 -0.002  0.001
price_displayed                -0.033    0.006 -5.685 0.000 -0.045 -0.022
fatigue                        -0.001    0.000 -2.826 0.

In [17]:
r_params = pd.DataFrame(r.params, columns = ["LMM"])
random_effects = pd.DataFrame(r.random_effects)
random_effects = random_effects.transpose()
random_effects = random_effects.rename(index = str, columns = {"group":"LMM"})

In [31]:
wof = [0, 0, -2, -2, -1, -1,-2, 3, 1, 1]

s = 0
for w in wof:
    s = s + w*(1/len(wof))
    
print (s)

-0.30000000000000004


In [32]:
import random

In [33]:
random.shuffle(wof)

In [34]:
print (wof)

[-2, 1, -2, 3, 0, -1, -1, -2, 1, 0]


In [ ]:
["#9B1003", "#3944BC", "#9B1003", "#051094", "#000000", "#D0312D", "#D0312D", "#9B1003","#3944BC", "#000000"]

